<a href="https://colab.research.google.com/github/gaurav-hegde/AnimeRecommendation/blob/main/AnimeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
anime = pd.read_csv('anime.csv')

Handling missing values

In [3]:
anime.fillna({'genre': '', 'rating': anime['rating'].mean(), 'type': 'Unknown'}, inplace=True)

In [4]:
anime['episodes'] = anime['episodes'].apply(lambda x: 1 if x == 'Unknown' else x)
anime['episodes'] = pd.to_numeric(anime['episodes'], errors='coerce')
anime['episodes'].fillna(anime['episodes'].mean(), inplace=True)

# Feature Engineering

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
anime['genre'] = anime['genre'].apply(lambda x: x.split(', ') if x != '' else [])

# One-hot encode genres
mlb = MultiLabelBinarizer()
genre_onehot = mlb.fit_transform(anime['genre'])
genre_df = pd.DataFrame(genre_onehot, columns=mlb.classes_)

# One-hot encode types
type_onehot = pd.get_dummies(anime['type'])

# Combine features into a single DataFrame
features = pd.concat([genre_df, type_onehot, anime[['rating', 'episodes']]], axis=1)

# Using Cosine Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(features)

# Recommendation Function

In [8]:
def recommend_anime(anime_name, anime_df, similarity_matrix, n=5):
    if anime_name not in anime_df['name'].values:
        return "Anime not found in the dataset."

    # Find the index of the given anime
    anime_idx = anime_df[anime_df['name'] == anime_name].index[0]

    # Get the pairwise similarity scores for all anime with the given anime
    similarity_scores = list(enumerate(similarity_matrix[anime_idx]))

    # Sort anime based on similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top n most similar anime
    top_anime_indices = [x[0] for x in similarity_scores[1:n+1]]

    # Return the names of the top n most similar anime
    return anime_df.iloc[top_anime_indices][['name', 'genre', 'type', 'rating', 'episodes']]

# User Input

In [9]:
def get_user_input_and_recommend(anime_df, similarity_matrix):
    # Take user input
    user_input = input("Enter the name of an anime you like: ")

    # Get recommendations
    recommendations = recommend_anime(user_input, anime_df, similarity_matrix, n=5)

    # Display recommendations
    if isinstance(recommendations, str):
        print(recommendations)
    else:
        print("Here are 5 anime similar to '{}':".format(user_input))
        for idx, row in recommendations.iterrows():
            print(f"{row['name']} (Type: {row['type']}, Rating: {row['rating']}, Episodes: {row['episodes']})")
            print(f"Genres: {', '.join(row['genre'])}")
            print("-" * 50)

# Example Usage

In [11]:
get_user_input_and_recommend(anime, similarity_matrix)

Enter the name of an anime you like: Bleach
Here are 5 anime similar to 'Bleach':
Keroro Gunsou (Type: TV, Rating: 7.93, Episodes: 358)
Genres: Comedy, Sci-Fi, Shounen
--------------------------------------------------
Kabatotto (Type: TV, Rating: 6.55, Episodes: 300)
Genres: Comedy
--------------------------------------------------
Atashin&#039;chi (Type: TV, Rating: 7.39, Episodes: 330)
Genres: Comedy, Slice of Life
--------------------------------------------------
Cookin&#039; Idol Ai! Mai! Main! (Type: TV, Rating: 6.56, Episodes: 305)
Genres: Comedy, Kids
--------------------------------------------------
Kochira Katsushikaku Kameari Kouenmae Hashutsujo (TV) (Type: TV, Rating: 7.82, Episodes: 373)
Genres: Comedy, Police, Slice of Life
--------------------------------------------------
